# Codigos de Prueba "Bridging Process"

---



> En este jupyter, basicamente trataremos de asemejar el proceso de puenteo que la empresa IQVIA realiza a traves de automatizaciónes con Visual Studio, pero en este caso en particular sera con "Transformers".

En la primera sección netamente esta enfocada en la importación de ciertas librerias basicas para generar DF's, a la par de una muy importante la cual es:

```
from transformers import BertTokenizer, BertModel
```



In [14]:
import pandas as pd
from google.colab import files
from transformers import BertTokenizer, BertModel
import torch
from scipy.spatial.distance import cosine

> Luego de eso generaremos una breve comparación de dos archivos CSV's, por lo cual debemos subirlos y generar una copia de estos de manera que no perdamos info. valiosa de los docs. originales

In [15]:
def cargar_archivos_csv():
    uploaded = files.upload()

    datos_csv = {}
    for filename in uploaded.keys():
        if filename.lower().endswith('.csv'):
            try:
                df = pd.read_csv(filename, delimiter=';', engine='python')
            except:
                df = pd.read_csv(filename)

            datos_csv[filename] = df
            print(f"Archivo permitido: {filename} | Extensión: csv")
        else:
            print(f"Archivo {filename} no permitido. Solo se aceptan archivos .csv")

    return datos_csv

In [21]:


# Cargar el modelo y el tokenizador preentrenado de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Función para convertir un texto en embeddings usando BERT
def text_to_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Promediar los embeddings
    return embeddings.squeeze().detach().numpy()  # Convertir a numpy array

# Función para comparar dos textos
def comparar_similitud_bert(text1, text2):
    embedding1 = text_to_embedding(text1)
    embedding2 = text_to_embedding(text2)
    similitud = 1 - cosine(embedding1, embedding2)  # Calcular similitud de coseno
    return similitud

# Función para encontrar las columnas relevantes en cada archivo
def obtener_columnas(df):
    # Intentar encontrar columnas relevantes, ignorando mayúsculas/minúsculas y espacios
    columnas = df.columns.str.strip().str.lower()

    col_nombre = None
    col_apellido = None
    col_direccion = None

    # Buscar columnas con nombres similares a "Nombre", "Apellido" y "Direccion"
    if 'nombre' in columnas:
        col_nombre = df.columns[columnas.get_loc('nombre')]
    if 'apellido' in columnas:
        col_apellido = df.columns[columnas.get_loc('apellido')]
    if 'direccion' in columnas or 'dirección' in columnas:  # También considerar "Dirección"
        col_direccion = df.columns[columnas.get_loc('direccion')] if 'direccion' in columnas else df.columns[columnas.get_loc('dirección')]

    return col_nombre, col_apellido, col_direccion

# Función para comparar registros de dos DataFrames
def comparar_registros(df_1, df_2):
    resultados = []

    # Obtener columnas relevantes
    nombre_1, apellido_1, direccion_1 = obtener_columnas(df_1)
    nombre_2, apellido_2, direccion_2 = obtener_columnas(df_2)

    if not (nombre_1 and apellido_1 and direccion_1):
        print("El primer archivo no contiene las columnas esperadas.")
        return

    if not (nombre_2 and apellido_2 and direccion_2):
        print("El segundo archivo no contiene las columnas esperadas.")
        return

    # Comparar cada registro del primer archivo con los del segundo
    for idx_1, fila_1 in df_1.iterrows():
        registro_nombre_1 = fila_1[nombre_1]
        registro_apellido_1 = fila_1[apellido_1]
        registro_direccion_1 = fila_1[direccion_1]

        mejor_similitud = 0
        mejor_fila = None

        for idx_2, fila_2 in df_2.iterrows():
            registro_nombre_2 = fila_2[nombre_2]
            registro_apellido_2 = fila_2[apellido_2]
            registro_direccion_2 = fila_2[direccion_2]

            # Calcular similitud para cada campo
            similitud_nombre = comparar_similitud_bert(str(registro_nombre_1), str(registro_nombre_2))
            similitud_apellido = comparar_similitud_bert(str(registro_apellido_1), str(registro_apellido_2))
            similitud_direccion = comparar_similitud_bert(str(registro_direccion_1), str(registro_direccion_2))

            # Promedio de similitud
            similitud_promedio = (similitud_nombre + similitud_apellido + similitud_direccion) / 3

            if similitud_promedio > mejor_similitud:
                mejor_similitud = similitud_promedio
                mejor_fila = fila_2

        # Determinar si es la misma persona según la similitud
        if mejor_similitud > 0.75:
            resultados.append({
                'Nombre_Archivo_1': registro_nombre_1,
                'Apellido_Archivo_1': registro_apellido_1,
                'Direccion_Archivo_1': registro_direccion_1,
                'Nombre_Archivo_2': mejor_fila[nombre_2] if mejor_fila is not None else 'No encontrado',
                'Apellido_Archivo_2': mejor_fila[apellido_2] if mejor_fila is not None else 'No encontrado',
                'Direccion_Archivo_2': mejor_fila[direccion_2] if mejor_fila is not None else 'No encontrado',
                'Similitud': f"{mejor_similitud * 100:.2f}%",
                'Misma_Persona': 'Yes' if mejor_similitud > 0.75 else 'No'
            })
        else:
            resultados.append({
                'Nombre_Archivo_1': registro_nombre_1,
                'Apellido_Archivo_1': registro_apellido_1,
                'Direccion_Archivo_1': registro_direccion_1,
                'Nombre_Archivo_2': 'No encontrado',
                'Apellido_Archivo_2': 'No encontrado',
                'Direccion_Archivo_2': 'No encontrado',
                'Similitud': "0.00%",
                'Misma_Persona': 'No'
            })

    return pd.DataFrame(resultados)

# Cargar los archivos CSV
datos_csv = cargar_archivos_csv()

# Verificar que se han cargado dos archivos CSV
if len(datos_csv) >= 2:
    archivo_1, archivo_2 = list(datos_csv.keys())[0:2]
    df_1 = datos_csv[archivo_1]
    df_2 = datos_csv[archivo_2]

    # Comparar los registros
    df_resultados = comparar_registros(df_1, df_2)

    if df_resultados is not None:
        # Guardar el DataFrame como un archivo CSV
        nombre_reporte = 'reporte_comparacion_doctores.csv'
        df_resultados.to_csv(nombre_reporte, index=False)

        # Descargar el archivo CSV
        files.download(nombre_reporte)

else:
    print("Se necesitan al menos dos archivos .csv para comparar.")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Saving Cliente.csv to Cliente (9).csv
Saving DMF.csv to DMF (9).csv
Archivo permitido: Cliente (9).csv | Extensión: csv
Archivo permitido: DMF (9).csv | Extensión: csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>